In [1]:
%matplotlib inline
# import necessary libraries and specify that graphs should be plotted inline. 

# from sklearn.tree import DecisionTreeClassifier
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import sklearn

import warnings
warnings.filterwarnings('ignore')

/Users/mrudula/opt/anaconda3/lib/python3.9/site-packages/pandas/core/computation/expressions.py:21: UserWarning: Pandas requires version '2.8.4' or newer of 'numexpr' (version '2.8.1' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED
/Users/mrudula/opt/anaconda3/lib/python3.9/site-packages/pandas/core/arrays/masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.2' currently installed).
  from pandas.core import (
/Users/mrudula/opt/anaconda3/lib/python3.9/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.26.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
# data loading
credit = pd.read_csv('cleaned_data.csv')

print(credit.head())

  Customer_ID     Month  Age Occupation  Annual_Income  Monthly_Inhand_Salary  \
0  CUS_0x1000   January   17     Lawyer       30625.94            2706.161667   
1  CUS_0x1000  February   17     Lawyer       30625.94            2706.161667   
2  CUS_0x1000     March   17     Lawyer       30625.94            2706.161667   
3  CUS_0x1000     April   17     Lawyer       30625.94            2706.161667   
4  CUS_0x1000       May   17     Lawyer       30625.94            2706.161667   

   Num_Bank_Accounts  Num_Credit_Card  Interest_Rate  Num_of_Loan  ...  \
0                6.0              5.0             27          2.0  ...   
1                6.0              5.0             27          2.0  ...   
2                6.0              5.0             27          2.0  ...   
3                6.0              5.0             27          2.0  ...   
4                6.0              5.0             27          2.0  ...   

  Credit_Mix  Outstanding_Debt  Credit_Utilization_Ratio  Credit_His

In [3]:
print(credit.columns)

Index(['Customer_ID', 'Month', 'Age', 'Occupation', 'Annual_Income',
       'Monthly_Inhand_Salary', 'Num_Bank_Accounts', 'Num_Credit_Card',
       'Interest_Rate', 'Num_of_Loan', 'Type_of_Loan', 'Delay_from_due_date',
       'Num_of_Delayed_Payment', 'Changed_Credit_Limit',
       'Num_Credit_Inquiries', 'Credit_Mix', 'Outstanding_Debt',
       'Credit_Utilization_Ratio', 'Credit_History_Age',
       'Payment_of_Min_Amount', 'Total_EMI_per_month',
       'Amount_invested_monthly', 'Payment_Behaviour', 'Monthly_Balance',
       'Credit_Score'],
      dtype='object')


# Converting Ordinal Categorical Variables to Numeric

In [4]:
credit['Credit_Score'].value_counts()

Credit_Score
Standard    53174
Poor        28998
Good        17828
Name: count, dtype: int64

In [5]:
# Credit Score column conversion to numeric column
size_mapping = {"Poor": 1, "Standard": 2, "Good": 3}

credit['Credit_Score'] = credit['Credit_Score'].map(size_mapping)

In [6]:
credit['Credit_Mix'].value_counts()

Credit_Mix
Standard    45848
Good        30384
Bad         23768
Name: count, dtype: int64

In [7]:
# Credit Mix column conversion to numeric column
size_mapping = {"Bad": 1, "Standard": 2, "Good": 3}

credit['Credit_Mix'] = credit['Credit_Mix'].map(size_mapping)

In [8]:
credit['Payment_of_Min_Amount'].value_counts()

Payment_of_Min_Amount
Yes    52326
No     35667
NM     12007
Name: count, dtype: int64

In [9]:
# Payment_of_Min_Amount column conversion to numeric column
size_mapping = {"Yes": 1, "No": 0, "NM": 0}

credit['Payment_of_Min_Amount'] = credit['Payment_of_Min_Amount'].map(size_mapping)

In [10]:
credit['Payment_Behaviour'].value_counts()

Payment_Behaviour
Low_spent_Small_value_payments      27588
High_spent_Medium_value_payments    19015
Low_spent_Medium_value_payments     15012
High_spent_Large_value_payments     14839
High_spent_Small_value_payments     12244
Low_spent_Large_value_payments      11302
Name: count, dtype: int64

In [11]:
# Payment Behaviour column conversion to numeric column
size_mapping = {"Low_spent_Small_value_payments": 1, "Low_spent_Medium_value_payments": 2, 
                "Low_spent_Large_value_payments": 3, "High_spent_Small_value_payments": 4, 
               "High_spent_Medium_value_payments": 5, "High_spent_Large_value_payments": 6}

credit['Payment_Behaviour'] = credit['Payment_Behaviour'].map(size_mapping)

In [12]:
columns_to_drop = ['Month', 'Occupation']
credit = credit.drop(columns=columns_to_drop)

In [13]:
credit.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 23 columns):
 #   Column                    Non-Null Count   Dtype  
---  ------                    --------------   -----  
 0   Customer_ID               100000 non-null  object 
 1   Age                       100000 non-null  int64  
 2   Annual_Income             100000 non-null  float64
 3   Monthly_Inhand_Salary     100000 non-null  float64
 4   Num_Bank_Accounts         100000 non-null  float64
 5   Num_Credit_Card           100000 non-null  float64
 6   Interest_Rate             100000 non-null  int64  
 7   Num_of_Loan               100000 non-null  float64
 8   Type_of_Loan              100000 non-null  object 
 9   Delay_from_due_date       100000 non-null  float64
 10  Num_of_Delayed_Payment    100000 non-null  float64
 11  Changed_Credit_Limit      100000 non-null  float64
 12  Num_Credit_Inquiries      100000 non-null  float64
 13  Credit_Mix                100000 non-null  in

In [14]:
import pandas as pd
from sklearn.preprocessing import OneHotEncoder

# One-hot encode the Type_of_Loan column
encoder = OneHotEncoder()
encoded_data = encoder.fit_transform(credit[['Type_of_Loan']]).toarray()

# Create a new dataframe from the encoded data
encoded_df = pd.DataFrame(encoded_data, columns=encoder.get_feature_names_out(['Type_of_Loan']))

# Concatenate the encoded dataframe with the original dataframe
credit = pd.concat([credit.drop('Type_of_Loan', axis=1), encoded_df], axis=1)

In [15]:
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

data = credit.iloc[:, 23:]
print(data.shape)

(100000, 6259)


In [16]:
pca = PCA(n_components=2)
df_pca = pca.fit_transform(data)
print(type(df_pca))

<class 'numpy.ndarray'>


In [17]:
df_pca_new = pd.DataFrame(df_pca)

In [18]:
credit_new = credit[['Age', 'Annual_Income', 'Monthly_Inhand_Salary',
       'Num_Bank_Accounts', 'Num_Credit_Card', 'Interest_Rate', 'Num_of_Loan', 
       'Delay_from_due_date', 'Num_of_Delayed_Payment',
       'Changed_Credit_Limit', 'Num_Credit_Inquiries', 'Credit_Mix',
       'Outstanding_Debt', 'Credit_Utilization_Ratio', 'Credit_History_Age',
       'Payment_of_Min_Amount', 'Total_EMI_per_month',
       'Amount_invested_monthly', 'Payment_Behaviour', 'Monthly_Balance',
       'Credit_Score']]

In [19]:
credit_new = pd.concat([credit_new, df_pca_new], axis=1)

In [20]:
credit_new.rename(
    columns={0:"PCA0", 1:"PCA1"}, inplace=True)

In [21]:
credit_new.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 23 columns):
 #   Column                    Non-Null Count   Dtype  
---  ------                    --------------   -----  
 0   Age                       100000 non-null  int64  
 1   Annual_Income             100000 non-null  float64
 2   Monthly_Inhand_Salary     100000 non-null  float64
 3   Num_Bank_Accounts         100000 non-null  float64
 4   Num_Credit_Card           100000 non-null  float64
 5   Interest_Rate             100000 non-null  int64  
 6   Num_of_Loan               100000 non-null  float64
 7   Delay_from_due_date       100000 non-null  float64
 8   Num_of_Delayed_Payment    100000 non-null  float64
 9   Changed_Credit_Limit      100000 non-null  float64
 10  Num_Credit_Inquiries      100000 non-null  float64
 11  Credit_Mix                100000 non-null  int64  
 12  Outstanding_Debt          100000 non-null  float64
 13  Credit_Utilization_Ratio  100000 non-null  fl